In [1]:
include("../../src/pure/Pnjl_pure.jl")

Trho (generic function with 1 method)

In [ ]:
T= 300.0
mu_B = 0.0
X0 = [-0.01, -0.01, -0.20, 0.8, 0.8]
phi = X0[1:3]
Phi1 = X0[4]
Phi2 = X0[5]
res = Omega(phi, Phi1, Phi2, T/hc, mu_B/hc,)

In [15]:
T = 10.0
rhos = 0.1:0.1:14.0
X0 = [-1.8,-1.8, -2.2, 0.01, 0.01, 320/hc, 320/hc, 320/hc]
rho_B = 0.1
data = zeros(length(rhos), 3)
data[:, 1] .= rhos
for (i, rho) in enumerate(rhos)
    #println("rho = ", rho)
    X0 = Trho(T/hc, rho, X0)
    phi = X0[1:3]
    Phi1 = X0[4]
    Phi2 = X0[5]
    mu_B = X0[6]*hc * 3
    P = -Omega(phi, Phi1, Phi2, T/hc, mu_B/hc)
    S = -dOmega_dT(phi, Phi1, Phi2, T/hc, mu_B/hc)
    rho_now = -dOmega_dmu_B(phi, Phi1, Phi2, T/hc, mu_B/hc)
    E = T/hc*S + mu_B/hc*rho_now - P
    data[i, 2] = P
    data[i, 3] = E
end
println("DONE")


DONE


In [16]:
using DataFrames, CSV, Plots
df = DataFrame(rho = data[:, 1], P = data[:, 2], E = data[:, 3])
CSV.write("EoS_Pnjl_pure_T10.csv", df)

"EoS_Pnjl_pure_T10.csv"